<a href="https://colab.research.google.com/github/Sankeerth55/AI-Career-Advisor-Agent/blob/main/AI_Career_Advisor_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install google-genai

In [60]:
!pip install -q U langgraph

In [61]:
!pip install langchain_google_genai

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [62]:
!pip install tavily-python

In [63]:
!pip install -q U langchain_community

In [64]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [65]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)
response = model.invoke("What is the capital of karnataka?")
print(response.content)

The capital of Karnataka is **Bangalore**.


In [66]:
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# Node 1: Extract detailed profile
def extract_profile(state):
    resume_text = state[-1].content
    prompt = (
        f"From the following resume, extract these sections separately:\n"
        f"1. Skills\n2. Education\n3. Experience\n\nResume:\n{resume_text}"
    )
    result = model.invoke([HumanMessage(content=prompt)]).content
    return state + [AIMessage(content=result)]

# Node 2: Generate roadmap
def generate_roadmap(state):
    extracted_profile = state[-2].content
    user_goal = state[-1].content
    prompt = (
        f"Use the following profile:\n{extracted_profile}\n\n"
        f"Now create a realistic 12-month career roadmap to achieve this target:\n{user_goal}"
    )
    roadmap = model.invoke([HumanMessage(content=prompt)]).content
    return state + [AIMessage(content=roadmap)]

# Node 3: Recommend roles and projects
def recommend_jobs_projects(state):
    profile = state[-3].content  # AIMessage with extracted profile
    goal = state[-1].content     # HumanMessage with goal
    prompt = (
        f"Profile:\n{profile}\n\n"
        f"Career Aspiration:\n{goal}\n\n"
        f"Suggest 3 suitable job roles and 3 open-source projects that align well."
    )
    output = model.invoke([HumanMessage(content=prompt)]).content
    return state + [AIMessage(content=output)]

# Build the graph
graph = MessageGraph()
graph.add_node("extract_profile", extract_profile)
graph.add_node("generate_roadmap", generate_roadmap)
graph.add_node("recommend_jobs_projects", recommend_jobs_projects)

graph.set_entry_point("extract_profile")
graph.add_edge("extract_profile", "generate_roadmap")
graph.add_edge("generate_roadmap", "recommend_jobs_projects")
graph.add_edge("recommend_jobs_projects", END)

agent = graph.compile()

# Invoke the graph
response = agent.invoke([
    HumanMessage(content="""
    Candidate Resume:
    Education: Bachelor of Technology in Computer Science
    Skills: Python, LangChain, SQL
    Experience: Internship at an AI-based startup
    """),
    HumanMessage(content="My goal is to become a Data Scientist within the next year.")
])

# Print the final result
for msg in response:
    print(f"{msg.type.upper()}:\n{msg.content}\n")

/tmp/ipython-input-312507512.py:38: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


HUMAN:

    Candidate Resume:
    Education: Bachelor of Technology in Computer Science
    Skills: Python, LangChain, SQL
    Experience: Internship at an AI-based startup
    

HUMAN:
My goal is to become a Data Scientist within the next year.

AI:
Okay, I can extract the requested sections from the provided resume.

**1. Skills:**

*   (None listed)

**2. Education:**

*   (None listed)

**3. Experience:**

*   (None listed)

AI:
Okay, given your profile (goal: Data Scientist in 12 months, and no existing skills, education, or experience listed), this is a realistic and ambitious 12-month roadmap. It assumes you are starting from scratch and willing to dedicate significant time and effort.  It focuses on building a solid foundation and demonstrating practical skills.  This roadmap prioritizes a blend of theoretical knowledge, hands-on projects, and networking.

**Phase 1: Foundations (Months 1-3): Building a Base**

*   **Month 1: Math & Statistics Fundamentals:**
    *   **Goal:** 

In [87]:
!pip install streamlit
!pip install pyngrok  # for tunneling, to access the Streamlit app externally
!pip install streamlit pyngrok PyPDF2

In [113]:
from pyngrok import ngrok
from pyngrok.exception import PyngrokNgrokHTTPError

try:
    # Disconnect any existing tunnels
    ngrok.kill()
    # Connect to the specified port
    public_url = ngrok.connect(8501)
    print("Your web app URL:", public_url)
except PyngrokNgrokHTTPError as e:
    print(f"An ngrok error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

ERROR:pyngrok.process.ngrok:t=2025-08-19T06:35:03+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-19T06:35:03+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

An unexpected error occurred: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.


In [114]:
from pyngrok import ngrok
import threading
import time

def run_streamlit():
    !streamlit run app.py --server.port 8501 --server.enableCORS false

# Run Streamlit in the background thread
threading.Thread(target=run_streamlit, daemon=True).start()

# Wait for Streamlit to start
time.sleep(10)

# Set ngrok auth token
ngrok.set_auth_token("30ijUvoQaYp6SRcS11A1rwAuXhQ_37Nn6K7uinS9XXEpzWS7w")

# Open tunnel on port 8501
public_url = ngrok.connect(8501, "http")
print("App live at:", public_url)

2025-08-19 06:35:07.284 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            


2025-08-19 06:35:07.697 Port 8501 is already in use
App live at: NgrokTunnel: "https://5441035ee33f.ngrok-free.app" -> "http://localhost:8501"
